# Reinforced learning, työllistyminen ja Suomen sosiaaliturva

Tässä tehdään laskelmat artikkelia varten. Käytössä on gym-ympäristö _unemployment-v1_ , johon on toteutettu yksityiskohtaisesti sosiaaliturvaa eri tiloissa.

In [1]:
# for Colab, install fin_benefits and unemployment-gym from Github
#!pip install -q git+https://github.com/ajtanskanen/benefits.git  
#!pip install -q git+https://github.com/ajtanskanen/econogym.git
#!pip install -q git+https://github.com/ajtanskanen/lifecycle-rl.git

# and then restart kernel
  
  # For a specific version:
#!pip install tensorflow==1.15
#!pip install stable-baselines==2.8
  
# restart kernel after running pip's

In [2]:
#import sys
#print(sys.path)
#sys.path.append('/usr/local/python3.7/site-packages')

Then load all modules and set parameters for simulations.

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from lifecycle_rl import Lifecycle

%matplotlib inline
%pylab inline

# varoitukset piiloon (Stable baseline ei ole vielä Tensorflow 2.0-yhteensopiva, ja Tensorflow 1.15 valittaa paljon)
# ei taida toimia piilottaminen
import warnings
warnings.filterwarnings('ignore')


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Populating the interactive namespace from numpy and matplotlib


In [4]:
# parameters for the simulation
# episode = 51 / 205 timesteps (1y/3m timestep)
pop_size=5_000 # size of the population to be simulated
size1=5_000_000 # number of timesteps in phase 1 training (callback not used)
size2=100_000_000 #0_000 # number of timesteps in phase 2 training (callback is used to save the best results)
size3=20_000_000 # number of timesteps in phase 1 training (callback not used) for policy changes
batch1=8 # size of minibatch in phase 1 as number of episodes
batch2=9_00  # size of minibatch in phase 1 as number of episodes
callback_minsteps=batch2 # how many episodes callback needs 
deterministic=True # use deterministic prediction (True) or probabilitic prediction (False)
mortality=False # include mortality in computations
randomness=True # include externally given, random state-transitions (parental leaves, disability, lay-offs) 
pinkslip=True # include lay-offs at 5 percent level each year
rlmodel='leaky_acktr' #'acktr' # use ACKTR algorithm # leaky_acktr?
twostage=False # ajataan kahdessa vaiheessa vai ei
if mortality:
    perusmalli_start='best/v2_malli_base_dev'
    perusmalli='best/v2_malli_base_dev'
    perusresults='results/v2_malli_base_dev_stoch'
    prefmalli='best/v2_malli_perus_prefnoise'
    prefresults='results/v2_perus_results_prefnoise'
else:
    perusmalli_start='best/v2_malli_baseline_nomort'
    perusmalli='best/v2_malli_base_nomort_dev'
    perusresults='results/v2_malli_base_nomort'
    baseline_results='results/v2_malli_baseline_nomort'
    prefmalli='best/v2_malli_perus_prefnoise_nomort'
    prefresults='results/v2_perus_results_prefnoise_nomort'
    
debug=False # jos True, niin ajetaan vain yhdellä prosessilla. Nopeampi debugata.
plotdebug=False # tulostetaanko rivi riviltä mitä tapahtuu

# Nykymalli 

Lasketaan työllisyysasteet nykymallissa.

In [ ]:
cc1=Lifecycle(env='unemployment-v2',minimal=False,mortality=mortality,perustulo=False,
              randomness=randomness,pinkslip=pinkslip,plotdebug=plotdebug)
cc1.run_results(debug=debug,steps1=size1,steps2=size2,pop=pop_size,deterministic=deterministic,
                train=True,predict=True,batch1=batch1,batch2=batch2,
                save=perusmalli_start,plot=False,cont=True,start_from=perusmalli_start,results=perusresults,
                callback_minsteps=callback_minsteps,rlmodel=rlmodel,twostage=twostage,
                learning_rate=0.25,learning_schedule='linear')

No mortality included
Parameters of lifecycle:
timestep 0.25
gamma 0.9793703613355593 (0.9200000000000003 per anno)
min_age 20
max_age 70
min_retirementage 63.5
max_retirementage 68
ansiopvraha_kesto300 300
ansiopvraha_kesto400 400
ansiopvraha_kesto500 500
ansiopvraha_toe 0.5
perustulo False
karenssi_kesto 0.25
mortality False
randomness True
include_putki True
include_pinkslip True
sigma_reduction True
plotdebug False

version 2
train...
phase 1
batch 1 learning rate 0.0625 scaled 0.0625 n_cpu 8




Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Please use `layer.__call__` method instead.








Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where






training...








---------------------------------
| explained_variance | 0.999    |
| fps                | 1358     |
| nupdates           | 1        |
| policy_entropy     | 1.17     |
| policy_loss        | -0.0186  |
| total_timesteps    | 1616   

---------------------------------
| explained_variance | 0.999    |
| fps                | 2534     |
| nupdates           | 50       |
| policy_entropy     | 1.15     |
| policy_loss        | -0.0173  |
| total_timesteps    | 80800    |
| value_loss         | 0.0163   |
---------------------------------
---------------------------------
| explained_variance | 0.999    |
| fps                | 2564     |
| nupdates           | 60       |
| policy_entropy     | 1.13     |
| policy_loss        | -0.0175  |
| total_timesteps    | 96960    |
| value_loss         | 0.022    |
---------------------------------
---------------------------------
| explained_variance | 0.999    |
| fps                | 2640     |
| nupdates           | 70       |
| policy_entropy     | 1.09     |
| policy_loss        | -0.00281 |
| total_timesteps    | 113120   |
| value_loss         | 0.0211   |
---------------------------------
---------------------------------
| explained_variance | 0.999    |
| fps         

---------------------------------
| explained_variance | 1        |
| fps                | 2825     |
| nupdates           | 320      |
| policy_entropy     | 1.1      |
| policy_loss        | 0.0114   |
| total_timesteps    | 517120   |
| value_loss         | 0.00783  |
---------------------------------
---------------------------------
| explained_variance | 0.999    |
| fps                | 2827     |
| nupdates           | 330      |
| policy_entropy     | 1.11     |
| policy_loss        | -0.0535  |
| total_timesteps    | 533280   |
| value_loss         | 0.0245   |
---------------------------------
---------------------------------
| explained_variance | 0.999    |
| fps                | 2829     |
| nupdates           | 340      |
| policy_entropy     | 1.12     |
| policy_loss        | -0.00919 |
| total_timesteps    | 549440   |
| value_loss         | 0.0138   |
---------------------------------
----------------------------------
| explained_variance | 0.999     |
| fps       

---------------------------------
| explained_variance | 0.998    |
| fps                | 2962     |
| nupdates           | 590      |
| policy_entropy     | 1.13     |
| policy_loss        | -0.0561  |
| total_timesteps    | 953440   |
| value_loss         | 0.0371   |
---------------------------------
---------------------------------
| explained_variance | 0.999    |
| fps                | 2967     |
| nupdates           | 600      |
| policy_entropy     | 1.19     |
| policy_loss        | -0.0432  |
| total_timesteps    | 969600   |
| value_loss         | 0.0173   |
---------------------------------
---------------------------------
| explained_variance | 0.999    |
| fps                | 2971     |
| nupdates           | 610      |
| policy_entropy     | 1.17     |
| policy_loss        | -0.00749 |
| total_timesteps    | 985760   |
| value_loss         | 0.0158   |
---------------------------------
---------------------------------
| explained_variance | 0.999    |
| fps         

---------------------------------
| explained_variance | 0.999    |
| fps                | 3011     |
| nupdates           | 860      |
| policy_entropy     | 1.14     |
| policy_loss        | -0.00598 |
| total_timesteps    | 1389760  |
| value_loss         | 0.0147   |
---------------------------------
---------------------------------
| explained_variance | 0.999    |
| fps                | 3015     |
| nupdates           | 870      |
| policy_entropy     | 1.13     |
| policy_loss        | -0.00579 |
| total_timesteps    | 1405920  |
| value_loss         | 0.0102   |
---------------------------------
---------------------------------
| explained_variance | 0.998    |
| fps                | 3010     |
| nupdates           | 880      |
| policy_entropy     | 1.13     |
| policy_loss        | -0.024   |
| total_timesteps    | 1422080  |
| value_loss         | 0.0332   |
---------------------------------
---------------------------------
| explained_variance | 0.999    |
| fps         

---------------------------------
| explained_variance | 0.999    |
| fps                | 3050     |
| nupdates           | 1130     |
| policy_entropy     | 1.14     |
| policy_loss        | -0.0146  |
| total_timesteps    | 1826080  |
| value_loss         | 0.0129   |
---------------------------------
---------------------------------
| explained_variance | 0.999    |
| fps                | 3051     |
| nupdates           | 1140     |
| policy_entropy     | 1.18     |
| policy_loss        | -0.0265  |
| total_timesteps    | 1842240  |
| value_loss         | 0.0144   |
---------------------------------
---------------------------------
| explained_variance | 1        |
| fps                | 3053     |
| nupdates           | 1150     |
| policy_entropy     | 1.09     |
| policy_loss        | -0.00471 |
| total_timesteps    | 1858400  |
| value_loss         | 0.00892  |
---------------------------------
----------------------------------
| explained_variance | 0.999     |
| fps       

---------------------------------
| explained_variance | 0.999    |
| fps                | 3074     |
| nupdates           | 1400     |
| policy_entropy     | 1.12     |
| policy_loss        | 0.0173   |
| total_timesteps    | 2262400  |
| value_loss         | 0.017    |
---------------------------------
---------------------------------
| explained_variance | 0.999    |
| fps                | 3072     |
| nupdates           | 1410     |
| policy_entropy     | 1.17     |
| policy_loss        | -0.0148  |
| total_timesteps    | 2278560  |
| value_loss         | 0.0114   |
---------------------------------
---------------------------------
| explained_variance | 0.999    |
| fps                | 3074     |
| nupdates           | 1420     |
| policy_entropy     | 1.22     |
| policy_loss        | -0.0642  |
| total_timesteps    | 2294720  |
| value_loss         | 0.022    |
---------------------------------
---------------------------------
| explained_variance | 0.999    |
| fps         

----------------------------------
| explained_variance | 0.999     |
| fps                | 3090      |
| nupdates           | 1670      |
| policy_entropy     | 1.1       |
| policy_loss        | -0.000428 |
| total_timesteps    | 2698720   |
| value_loss         | 0.0128    |
----------------------------------
---------------------------------
| explained_variance | 1        |
| fps                | 3090     |
| nupdates           | 1680     |
| policy_entropy     | 1.14     |
| policy_loss        | 0.00882  |
| total_timesteps    | 2714880  |
| value_loss         | 0.00738  |
---------------------------------
---------------------------------
| explained_variance | 1        |
| fps                | 3091     |
| nupdates           | 1690     |
| policy_entropy     | 1.13     |
| policy_loss        | -0.0152  |
| total_timesteps    | 2731040  |
| value_loss         | 0.00701  |
---------------------------------
---------------------------------
| explained_variance | 0.999    |
| fps

---------------------------------
| explained_variance | 0.999    |
| fps                | 3091     |
| nupdates           | 1940     |
| policy_entropy     | 1.18     |
| policy_loss        | 0.00642  |
| total_timesteps    | 3135040  |
| value_loss         | 0.0121   |
---------------------------------


In [ ]:
cc1=Lifecycle(env='unemployment-v2',minimal=False,mortality=mortality,perustulo=False,
              randomness=randomness,pinkslip=pinkslip,plotdebug=plotdebug)
cc1.render(load=perusresults,figname='v2_')

In [ ]:
for t in set([0,10,20,30,40]):
    cc1.episodestats.plot_saldist(t)

In [ ]:
cc1=Lifecycle(env='unemployment-v2',minimal=False,mortality=mortality,perustulo=False,
              randomness=randomness,pinkslip=pinkslip,plotdebug=plotdebug)
cc1.load_sim(perusresults)
cc1.episodestats.test_salaries()

In [ ]:
cc1=Lifecycle(env='unemployment-v2',minimal=False,mortality=mortality,perustulo=False,
              randomness=randomness,pinkslip=pinkslip,plotdebug=plotdebug)

cc1.env.unit_test_code_decode()

# IRR

In [ ]:
if True:
    cc1=Lifecycle(env='unemployment-v2',minimal=False,mortality=mortality,perustulo=False,
                  randomness=randomness,pinkslip=pinkslip,plotdebug=plotdebug)
    cc1.load_sim(perusresults)
    cc1.episodestats.plot_irr(figname='kela')

In [ ]:
#cc1=Lifecycle(env='unemployment-v2',minimal=False,mortality=mortality,perustulo=False,
#              randomness=randomness,pinkslip=pinkslip,plotdebug=plotdebug)
#cc1.env.test_comp_npv()

# test

In [ ]:
cc1=Lifecycle(env='unemployment-v2',minimal=False,mortality=mortality,perustulo=False,
              randomness=randomness,pinkslip=pinkslip,plotdebug=plotdebug)
cc1.load_sim(perusresults)
cc1.episodestats.get_vanhempainvapaat()
cc1.episodestats.get_vanhempainvapaat_md()

In [ ]:
cc1=Lifecycle(env='unemployment-v2',minimal=False,mortality=mortality,perustulo=False,
              randomness=randomness,pinkslip=pinkslip,plotdebug=plotdebug)
cc1.load_sim(perusresults)
cc1.episodestats.plot_emp_by_gender()